In [ ]:
#| default_exp decoders

# Decoders
> Various utilities to decode MARIS dataset from `NetCDF`.

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export
from pathlib import Path
from netCDF4 import Dataset
import pandas as pd
import numpy as np
from fastcore.basics import patch, store_attr
import fastcore.all as fc
from typing import Dict, Callable

from marisco.configs import (
    NC_DTYPES, 
    NC_VARS, 
    OR_VARS,
    NC_DIM,
    NC_GROUPS,
    SMP_TYPE_LUT,
    lut_path, 
    Enums,
    nc_tpl_path,
    get_time_units
)

from marisco.callbacks import (
    DecodeTimeCB
    )

## Convert NetCDF to OpenRefine CSV


Maris NetCDF files can be converted to OpenRefine CSV files. The OpenRefine CSV files are compatible with the [OpenRefine](https://openrefine.org/) data cleaning tool which are used during the MARIS data cleaning process before loading into the MARIS database.


In [ ]:
#| exports
class NetCDFDecoder:
    """Decode MARIS NetCDF files to human readable formats."""
    def __init__(self, 
                 dfs: Dict[str, pd.DataFrame], 
                 fname_in: str,  # Path to NetCDF file
                 dest_out: str, 
                 output_format:str, 
                 remap_vars: Dict[str, str],
                 verbose: bool=False
                ):
        fc.store_attr()        

In [ ]:
#| exports
@patch
def process_groups(self: NetCDFDecoder):
    """Process all groups in the dataset."""
    for group_name, df in self.dfs.items():
        self.process_group(group_name, df, self.remap_vars)

In [ ]:
#| exports
@patch
def process_group(self: NetCDFDecoder, group_name: str, df: pd.DataFrame, remap_vars: Dict[str, str]):
    """Process a single group, mapping column names using remap_vars."""
    # Map column names using remap_vars
    df.columns = [remap_vars.get(col, col) for col in df.columns]


In [ ]:
#| exports
@patch
def save_dataframes(self: NetCDFDecoder):
    """
    Save DataFrames to files in the specified format.
    
    Parameters:
        dest_path (str, optional): Base path for output files, without extension.
            If None, uses self.dest_fname's path without extension.
        output_format (str): Format to save files in. Options:
            - 'csv': Comma-separated values
            - 'excel': Excel spreadsheet (one sheet per group)
            - 'json': JSON format
            - 'parquet': Apache Parquet format
            - 'hdf': HDF5 format
            - 'pickle': Python pickle format
            - 'feather': Feather format
            - 'stata': Stata format
    """
    # Get base path without extension
    if self.dest_out is None:
            raise ValueError("No destination path provided")
    else:
        base_path = str(Path(self.dest_out).with_suffix(''))
    
    # Handle formats that combine all groups in one file
    if self.output_format == 'excel':
        output_path = f"{base_path}.xlsx"
        with pd.ExcelWriter(output_path) as writer:
            for group_name, df in self.dfs.items():
                df.to_excel(writer, sheet_name=group_name, index=False)
                if self.verbose:
                    print(f"Saved {group_name} to sheet in {output_path}")
    
    elif self.output_format == 'hdf':
        output_path = f"{base_path}.h5"
        with pd.HDFStore(output_path) as store:
            for group_name, df in self.dfs.items():
                store[group_name] = df
                if self.verbose:
                    print(f"Saved {group_name} to group in {output_path}")
    
    # Handle formats that create separate files for each group
    else:
        format_extensions = {
            'csv': '.csv',
            'json': '.json',
            'parquet': '.parquet',
            'pickle': '.pkl',
            'feather': '.feather',
            'stata': '.dta'
        }
        
        if self.output_format not in format_extensions:
            raise ValueError(f"Unsupported output format: {self.output_format}. Supported formats: {format_extensions.keys()}")
            
        extension = format_extensions[self.output_format]
        save_methods = {
            'csv': lambda df, path: df.to_csv(path, index=False),
            'json': lambda df, path: df.to_json(path),
            'parquet': lambda df, path: df.to_parquet(path),
            'pickle': lambda df, path: df.to_pickle(path),
            'feather': lambda df, path: df.to_feather(path),
            'stata': lambda df, path: df.to_stata(path)
        }
        
        for group_name, df in self.dfs.items():
            output_path = f"{base_path}_{group_name}{extension}"
            save_methods[self.output_format](df, output_path)
            
            if self.verbose:
                print(f"Saved {group_name} to {output_path}")

In [ ]:
#| exports
@patch
def decode(self: NetCDFDecoder):
    "  Decode NetCDF to Human readable files."
    # Funvtion to rename the columns. 
    self.process_groups()
    self.save_dataframes()
    
    return self.dfs
    

In [ ]:
#|eval: false

df_seawater = pd.DataFrame({
    'ID': [0, 1, 2], 
    'LON': [141, 142, 143], 
    'LAT': [37.3, 38.3, 39.3], 
    'TIME': [1234, 1235, 1236], 
    'NUCLIDE': [1, 2, 3],
    'VALUE': [0.1, 1.1, 2.1], 
    'AREA': [2374, 2379, 2401],
    })

df_biota = pd.DataFrame({
    'ID': [0, 1, 2, 3], 
    'LON': [141, 142, 143, 144], 
    'LAT': [37.3, 38.3, 39.3, 40.3], 
    'TIME': [1234, 1235, 1236, 1237], 
    'NUCLIDE': [1, 2, 3, 3],
    'VALUE': [0.1, 1.1, 2.1, 3.1], 
    'SPECIES': [1, 2, 3, 3]
    })
dfs = {'SEAWATER': df_seawater, 'BIOTA': df_biota}


In [ ]:
#|eval: false
fname = Path('../../_data/output/100-HELCOM-MORS-2024.nc')

decoder = NetCDFDecoder( 
                        dfs=dfs,
                        fname_in=fname,  
                        dest_out=fname.with_suffix(''),
                        output_format='csv',
                        remap_vars=OR_VARS,
                        verbose=True
                 )
decoder.decode()


Saved SEAWATER to ../../_data/output/100-HELCOM-MORS-2024_SEAWATER.csv
Saved BIOTA to ../../_data/output/100-HELCOM-MORS-2024_BIOTA.csv


{'SEAWATER':    ID  longitude  latitude  begperiod  nuclide_id  activity  area
 0   0        141      37.3       1234           1       0.1  2374
 1   1        142      38.3       1235           2       1.1  2379
 2   2        143      39.3       1236           3       2.1  2401,
 'BIOTA':    ID  longitude  latitude  begperiod  nuclide_id  activity  species_id
 0   0        141      37.3       1234           1       0.1           1
 1   1        142      38.3       1235           2       1.1           2
 2   2        143      39.3       1236           3       2.1           3
 3   3        144      40.3       1237           3       3.1           3}